# Session 11 - Generative language models for zero-shot learning

In [43]:
from transformers import TFAutoModelForSeq2SeqLM, AutoTokenizer
import os
import pandas as pd

We're going to be working with ```FLAN-T5```, a text-to-text model developed by Google. ```FLAN-T5``` is based on ```T5```, which we saw in the lecture, but it has been further finetuned on a range of common text-to-text tasks. This means that it can already perform a lot of the kinds of tasks that people use generative language models for. You can read more in the paper here: [https://arxiv.org/pdf/2210.11416.pdf](https://arxiv.org/pdf/2210.11416.pdf)

We load the model from ```huggingface```. We're here using the ```Large``` version, but you can try the other sizes if you want. The ```Large``` version is already 3.4GB, and even larger models will take a long time to download and to run - but they should also see a marked performance improvement.
 
You can read more about the available models here: [https://huggingface.co/docs/transformers/model_doc/flan-t5](https://huggingface.co/docs/transformers/model_doc/flan-t5)

In [44]:
model = TFAutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large")

/home/ucloud/.local/lib/python3.10/site-packages/tf_keras/src/initializers/initializers.py:121: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


We also download and initalize the pretrained tokenizer that fits with our model:

In [45]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

## Prompt engineering

Our goal is to use the knowledge of language that FLAN-T5 has already acquired during training and to use that knowledge in different domains *without any further fine-tuning*. This is an example of what is called *zero-shot* learning.

In order for zero-shot learning to be successful, our prompts need to be carefully designed. FLAN-T5 (and similar models) are a bit less flexible than, for example, ChatGPT.

In [46]:
# classification
# prompt = "classify the following text as positive or negative: I absolutely hated this movie"

# translation
# prompt = "translate from English to French: how old are you?"

# question answering
# prompt = "answer the following question: how is cheese made?"

# named entity recognition
prompt = "find all location entities in this text: Ross comes from Scotland"

We then pass our text prompt to the tokenizer, defing some extra arguments such as the ```max_length``` of our input (anything longer than this will be truncated):

In [47]:
inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


We then pass all of our input prompt tokens to the model and use than to generate an appropriate output from what FLAN-T5 has learned during training.

In [48]:
outputs = model.generate(**inputs)

/home/ucloud/.local/lib/python3.10/site-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [49]:
print(tokenizer.batch_decode(outputs, 
                            skip_special_tokens=True))

['Scotland']


We can also do this a bit more cleverly by using a single prompt plus a F-string:

In [50]:
prompt = f"classify the following text as positive or negative: {input_text}"

This means we could, for example, write functions for specific tasks:

In [51]:
def classifier(input_text:str) -> str:
    prompt = f"classify the following text as positive or negative: {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

In [52]:
classifier("I absolutely hated this movie!")

['negative']


## Tasks

Look through previous notebooks, exercises, and datasets from Language Analytics so far this semester. In small groups, try using either ```Flan-T5``` and **ChatGPT** (or both) try to solve those problems using generative language models.

So that would mean, for example:

    - Grammatical analysis
    - Named entity recognition/extraction
    - Classification
    - Topic modelling

As an illustrative example: try using Flan-T5 to perform classification on the Fake or Real News dataset. How does it perform on ground truth? Is it better or worse than the other models we've seen? Be creative!


In [58]:
filepath = os.path.join("..", "..", "cds-language","data","fake_or_real_news.csv")

df = pd.read_csv(filepath)

input_text = df['text']

# print(input_text[0])

def classifier(input_text:str) -> str:
    prompt = f"classify the following text as real or fake: {input_text}"
    inputs = tokenizer(prompt, 
                    max_length = 200,
                    return_tensors="tf")
    outputs = model.generate(**inputs)
    print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

classifier(input_text[1])

y_labels = df['label']
print(y_labels[1])

/home/ucloud/.local/lib/python3.10/site-packages/transformers/generation/tf_utils.py:837: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length.  recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['real']
FAKE


: 